In [20]:
!pip install lark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**checking whether program contains atleat one loop using lark tool**

In [21]:
from lark import Lark

# Define the grammar
grammar = """
start: statement*

statement: loop | assign

loop: "for"  ID "in" range  block

assign: ID ":=" expression ";"

block: "{" statement* "}"

range: expression ".." expression

expression: ID | NUMBER | expression ( "+" | "-" ) expression

ID: /[a-zA-Z][a-zA-Z0-9]*/

NUMBER: /\d+/
%import common.WS
%ignore WS
"""

# Create the parser
parser = Lark(grammar)

code = '''
for i in 1..10
    {
          x := 0; x := x + 1;
   }'''

# Parse the code
tree = parser.parse(code)

# Find all loop statements
loops = tree.find_data('loop')

if loops:
    print("The code contains at least one loop.")
else:
    print("The code doesn't contain any loops.")

The code contains at least one loop.


**checking for nested loop using AST**

In [25]:
import ast

def check_nested_loops(tree):
    for node in ast.walk(tree):
        if isinstance(node, (ast.For, ast.While)):
            for child in ast.iter_child_nodes(node):
                if isinstance(child, (ast.For, ast.While)):
                    print("Nested loops found!")
                    return
    print("No nested loops found.")

code = '''
n=input()
m=input()
for i in range(n):
    for j in range(m):
        print(i * j)
'''

tree = ast.parse(code)
print(tree)
check_nested_loops(tree)


Nested loops found!


**finding the dependent variables using AST**

In [23]:

import ast

code = '''
x = 2
y = x + 3
z = x * y

def my_func(a, b):
    c = a + b
    return c
'''

tree = ast.parse(code)

deps = {}
for node in ast.walk(tree):
    if isinstance(node, ast.Assign):
        for target in node.targets:
            if isinstance(target, ast.Name):
                dep = [n.id for n in ast.walk(node) if isinstance(n, ast.Name) and n.id != target.id]
                deps[target.id] = dep
print(deps)


{'x': [], 'y': ['x'], 'z': ['x', 'y'], 'c': ['a', 'b']}


**Finding all variables in a program using AST**

In [27]:
import ast

def check_variable_dependencies(tree):
    variables = set()
    for node in ast.walk(tree):
        if isinstance(node, ast.Name):
            variables.add(node.id)
        elif isinstance(node, (ast.For, ast.While, ast.FunctionDef, ast.ClassDef)):
            for child in ast.iter_child_nodes(node):
                if isinstance(child, ast.Assign):
                    for target in child.targets:
                        if isinstance(target, ast.Name):
                            variables.add(target.id)
    print("Variables:", variables)

code = '''
x = 2
y = x + 3
z = x * y

def my_func(a, b):
    c = a + b
    return c
'''
tree = ast.parse(code)
check_variable_dependencies(tree)


Variables: {'b', 'c', 'z', 'y', 'x', 'a'}
